In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# plyaer data load 및 정리(프로필)
player_df = pd.read_csv('../data/new/players_all.csv')
player_df = player_df.rename(columns={'Unnamed: 0':'Id'}).drop(['No.', 'Id'], axis=1)

In [3]:
# 1xbet data load 및 정리(스텟)
xbet_df = pd.read_csv('../data/new/1xbet_all.csv')
xbet_df = xbet_df.drop(['Team','Age','Position'], axis=1).rename(columns={'Drb_x':'Drb_Off', 'Drb_y':'Drb_Def'})

In [4]:
# understat data load 및 정리(스텟)
understat_df = pd.read_csv('../data/new/understat_all.csv')
understat_df = understat_df.drop(['No','Team'], axis=1)

In [5]:
# capology load(연봉)
capology_df = pd.read_csv('../data/new/capology_all.csv')

In [6]:
# 프로필 + 연봉
eda_df = player_df.merge(capology_df[['Weekly Salary','Base Salary','ADJ Salary','Name', 'year']], how='left', left_on=['Name', 'year'], right_on=['Name', 'year'])
eda_df = eda_df[~eda_df.duplicated(['year', 'Name'], keep='first')] # 중복 제거

In [7]:
# 프로필 + 연봉 + 스텟(understat)
eda_df = eda_df.merge(understat_df, how='left', left_on=['Name', 'year'], right_on=['Name', 'year'])

In [8]:
# 프로필 + 연봉 + 스텟(understat) + 스텟(1xbet)
eda_df = eda_df.merge(
    xbet_df[[
        'Name', 'year', 'SpG', 'KeyP', 'Drb_Off', 'Fouled', 'Off',
        'Disp', 'UnsTch', 'Rating', 'Tackles', 'Inter','Fouls',
        'Offsides', 'Clear', 'Drb_Def', 'Blocks', 'AvgP', 'PS%']],
    how='left',
    left_on=['Name', 'year'],
    right_on=['Name', 'year']
)

In [9]:
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4756 entries, 0 to 4755
Data columns (total 42 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           4756 non-null   int64  
 1   Name           4756 non-null   object 
 2   Age            4756 non-null   int64  
 3   Team           4756 non-null   object 
 4   Position       4756 non-null   object 
 5   Weekly Salary  4598 non-null   float64
 6   Base Salary    4598 non-null   float64
 7   ADJ Salary     4598 non-null   float64
 8   Apps           4744 non-null   float64
 9   Min            4744 non-null   float64
 10  G              4744 non-null   float64
 11  NPG            4744 non-null   float64
 12  A              4744 non-null   float64
 13  xG             4744 non-null   float64
 14  NPxG           4744 non-null   float64
 15  xA             4744 non-null   float64
 16  xGChain        4744 non-null   float64
 17  xGBuildup      4744 non-null   float64
 18  xG90    

In [10]:
eda_df = eda_df.dropna()

In [11]:
eda_df.describe()

,year,Age,Weekly Salary,Base Salary,ADJ Salary,Apps,Min,G,NPG,A,...,Rating,Tackles,Inter,Fouls,Offsides,Clear,Drb_Def,Blocks,AvgP,PS%
count,4588.000000,4588.000000,4588.000000,4.588000e+03,4.588000e+03,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000,...,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000,4588.000000
mean,2017.934394,25.914560,53921.124237,2.803899e+06,3.076834e+06,20.581953,1466.584350,1.964255,1.814080,1.389494,...,6.649458,1.124752,0.780236,0.698120,0.125679,1.531114,0.607180,0.223566,29.003466,76.909555
std,2.604069,4.205339,52782.048685,2.744666e+06,2.942733e+06,11.491688,1032.205164,3.519175,3.177971,2.191754,...,0.358759,0.839077,0.703740,0.463213,0.229381,1.754969,0.481147,0.294067,16.436788,12.415437
min,2014.000000,15.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000,...,4.950000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,23.000000,20000.000000,1.040000e+06,1.252784e+06,11.000000,532.500000,0.000000,0.000000,0.000000,...,6.397240,0.500000,0.184211,0.379077,0.000000,0.333333,0.250000,0.000000,17.000000,72.727273
50%,2018.000000,26.000000,40000.000000,2.080000e+06,2.301310e+06,22.000000,1390.000000,1.000000,1.000000,1.000000,...,6.649188,1.000000,0.646110,0.676471,0.000000,0.864865,0.533333,0.108108,27.285714,79.316827
75%,2020.000000,29.000000,70000.000000,3.640000e+06,4.071549e+06,31.000000,2326.250000,2.000000,2.000000,2.000000,...,6.883926,1.629630,1.200000,1.000000,0.153846,2.088904,0.875000,0.333333,38.439236,84.322658
max,2022.000000,41.000000,600000.000000,3.120000e+07,3.367168e+07,38.000000,3420.000000,36.000000,31.000000,20.000000,...,8.340000,7.000000,4.216216,4.000000,2.000000,11.500000,4.000000,2.052632,90.411765,100.000000
